In [36]:
!pip install albumentations

In [37]:
!pip install opencv-python

In [38]:
import tensorflow as tf### models
import numpy as np### math computations
import seaborn as sns### visualizations
import matplotlib.pyplot as plt### plotting bar chart
import pathlib
import io
import xml.etree.ElementTree as ET
import os
import shutil
import cv2
import time
import random
from PIL import Image
import albumentations as A
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape, LeakyReLU)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d huanghanchina/pascal-voc-2012

mkdir: cannot create directory ‘/root/.kaggle’: File exists
pascal-voc-2012.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip "/content/pascal-voc-2012.zip" -d "/content/pascal-voc-2012/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001829.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001874.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001876.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001882.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001885.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001895.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001896.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001926.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001966.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC2012/SegmentationClass/2008_001971.png  
  inflating: /content/pascal-voc-2012/voc2012/VOC

In [3]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [4]:
train_images='/notebooks/pascal-voc-2012/VOC2012/JPEGImages/'
train_maps='/notebooks/pascal-voc-2012/VOC2012/Annotations/'

val_images='/notebooks/pascal-voc-2012/VOC2012/ValJPEGImages/'
val_maps='/notebooks/pascal-voc-2012/VOC2012/ValAnnotations/'

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2
N_CLASSES=len(classes)
H,W =224,224
SPLIT_SIZE=H//32
print(SPLIT_SIZE)
N_EPOCHS=135
BATCH_SIZE=32

7


In [5]:
!mkdir /notebooks/pascal-voc-2012/VOC2012/ValJPEGImages/
!mkdir /notebooks/pascal-voc-2012/VOC2012/ValAnnotations/

mkdir: cannot create directory ‘/notebooks/pascal-voc-2012/VOC2012/ValJPEGImages/’: File exists
mkdir: cannot create directory ‘/notebooks/pascal-voc-2012/VOC2012/ValAnnotations/’: File exists


In [51]:
for name in val_list:
  shutil.move(train_maps+name[:-3]+"xml", val_maps+name[:-3]+"xml")

FileNotFoundError: [Errno 2] No such file or directory: '/notebooks/pascal-voc-2012/VOC2012/Annotations/2007_000027.xml'

In [52]:
for name in val_list:
  shutil.move(train_images+name, val_images+name)

# Data Preparation

In [5]:
class_dict = {c: i for i,c in enumerate(classes)}
class_dict

{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'bottle': 4,
 'bus': 5,
 'car': 6,
 'cat': 7,
 'chair': 8,
 'cow': 9,
 'diningtable': 10,
 'dog': 11,
 'horse': 12,
 'motorbike': 13,
 'person': 14,
 'pottedplant': 15,
 'sheep': 16,
 'sofa': 17,
 'train': 18,
 'tvmonitor': 19}

In [6]:
def preprocess_xml(filename):
  tree = ET.parse(filename)
  root = tree.getroot()
  size_tree = root.find('size')
  height = float(size_tree.find('height').text)
  width = float(size_tree.find('width').text)
  bounding_boxes = []
  objects_tree = root.findall('object')
  for obj in objects_tree:
    for bounding_box in obj.iter('bndbox'):
      xmin = (float(bounding_box.find('xmin').text))
      ymin = (float(bounding_box.find('ymin').text))
      xmax = (float(bounding_box.find('xmax').text))
      ymax = (float(bounding_box.find('ymax').text))
      break
    class_name = obj.find('name').text
    x_center_norm = (xmin+xmax)/(2*width)
    y_center_norm = (ymin+ymax)/(2*height)
    width_norm = (xmax-xmin)/width
    height_norm = (ymax-ymin)/height

    bbox=[x_center_norm,y_center_norm,width_norm,height_norm,class_dict[class_name]]
    bounding_boxes.append(bbox)
  return tf.convert_to_tensor(bounding_boxes)

In [7]:
preprocess_xml(val_maps+"2007_000032.xml")

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.479     ,  0.4644128 ,  0.542     ,  0.37366548,  0.        ],
       [ 0.33      ,  0.37544483,  0.128     ,  0.12455516,  0.        ],
       [ 0.408     ,  0.727758  ,  0.036     ,  0.17437722, 14.        ],
       [ 0.07      ,  0.7597865 ,  0.036     ,  0.17437722, 14.        ]],
      dtype=float32)>

In [8]:
def generate_output(bboxes):
  output_labels = np.zeros(shape=(SPLIT_SIZE,SPLIT_SIZE,N_CLASSES+5))
  for bbox in bboxes:
    grid_x = bbox[0]*SPLIT_SIZE
    grid_y = bbox[1]*SPLIT_SIZE
    i = int(grid_x)
    j = int(grid_y)
    output_labels[i,j,:5] = [1.,grid_x%1,grid_y%1,bbox[2],bbox[3]]
    output_labels[i,j,5+int(bbox[4])] = 1
  return tf.convert_to_tensor(output_labels,tf.float32)

In [9]:
generate_output(preprocess_xml(val_maps+"2007_000032.xml"))[3][3]

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([1.        , 0.35299993, 0.25088978, 0.542     , 0.37366548,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ],
      dtype=float32)>

In [10]:
im_paths=[]
xml_paths=[]

val_im_paths=[]
val_xml_paths=[]


for i in os.listdir(train_maps):
  im_paths.append(train_images+i[:-3]+'jpg')
  xml_paths.append(train_maps+i)

for i in os.listdir(val_maps):
  val_im_paths.append(val_images+i[:-3]+'jpg')
  val_xml_paths.append(val_maps+i)

print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))

17061 17061
64 64


In [11]:
train_dataset=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_dataset=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))

In [13]:
for i in val_dataset.take(1):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'/notebooks/pascal-voc-2012/VOC2012/ValJPEGImages/2007_001311.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'/notebooks/pascal-voc-2012/VOC2012/ValAnnotations/2007_001311.xml'>)


In [12]:
def get_imbboxes(im_path,xml_path):
  img = tf.io.decode_jpeg(tf.io.read_file(im_path))
  img = tf.cast(tf.image.resize(img,size=(H,W)),dtype=tf.float32)
  bboxes = tf.numpy_function(func=preprocess_xml, inp=[xml_path],Tout=tf.float32)
  return img, bboxes

In [13]:
train_dataset=train_dataset.map(get_imbboxes)
val_dataset=val_dataset.map(get_imbboxes)

In [14]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor(
[[0.771      0.62266666 0.326      0.568      8.        ]
 [0.279      0.39466667 0.258      0.368      8.        ]], shape=(2, 5), dtype=float32)


# Data Augmentation by Albumentation

In [15]:
transforms = A.Compose([
    A.Resize(H,W),
    A.RandomCrop(
         width=np.random.randint(int(0.9*W),W),
         height=np.random.randint(int(0.9*H),H), p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4,p=0.5),
    A.HorizontalFlip(p=0.5,),
    A.Resize(H,W),

], bbox_params=A.BboxParams(format='yolo', ))

In [16]:
def aug_albument(image,bboxes):
  augmented=transforms(image=image,bboxes=bboxes)
  return [tf.convert_to_tensor(augmented["image"],dtype=tf.float32),
          tf.convert_to_tensor(augmented["bboxes"],dtype=tf.float32)]

In [17]:
def process_data(image,bboxes):
    aug= tf.numpy_function(func=aug_albument, inp=[image,bboxes], Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]

In [18]:
train_dataset=train_dataset.map(process_data)

In [20]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor(
[[0.22899997 0.62266666 0.326      0.568      8.        ]
 [0.72099996 0.39466667 0.258      0.368      8.        ]], shape=(2, 5), dtype=float32)


In [22]:
cv2.imwrite('out_1.jpg',np.array(i))

True

In [21]:
def preprocess_augment(img,labels):
  img = tf.image.random_brightness(img, max_delta=50.)
  img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
  img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
  img = tf.clip_by_value(img, 0, 255)
  labels=tf.numpy_function(func=generate_output, inp=[labels], Tout=(tf.float32))
  return img,labels

In [22]:
def preprocess(img,labels):
  img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)
  labels=tf.numpy_function(func=generate_output, inp=[labels], Tout=(tf.float32))
  return img,labels

In [23]:
train_dataset=train_dataset.map(preprocess_augment)
val_dataset=val_dataset.map(preprocess)

In [24]:
train_dataset=(
  train_dataset.
  batch(BATCH_SIZE).
  prefetch(tf.data.AUTOTUNE)
)

val_dataset=(
  val_dataset.
  batch(BATCH_SIZE).
  prefetch(tf.data.AUTOTUNE)
)


In [27]:
t1 = np.array([[1, 2, 3], [4, 5, 6]])
t2 = np.array([[7, 8, 9], [10, 11, 12]])
a = tf.concat([tf.expand_dims(t1[...,0],axis=0),tf.expand_dims(t2[...,2],axis=0)],axis=0)
b = tf.stack([t1[...,0],t2[...,0]],axis=0)
c = tf.transpose(a)
print(a)
print(b)
print(c)
mask =tf.cast(tf.math.greater(10,12),dtype=tf.int32)
print(mask)

tf.Tensor(
[[ 1  4]
 [ 9 12]], shape=(2, 2), dtype=int64)
tf.Tensor(
[[ 1  4]
 [ 7 10]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 1  9]
 [ 4 12]], shape=(2, 2), dtype=int64)
tf.Tensor(0, shape=(), dtype=int32)


In [28]:
for i,j in train_dataset.take(1):
  print(i.shape,j[...,0][0,0,0])
  target = j[...,0]
  y_target_extract = tf.gather_nd(j, tf.where(target[:]==1))
  print(y_target_extract)
  indices = tf.where(target[:]==1)
  print(j[indices[0][0],indices[0][1],indices[0][2]])
  print(indices)
  rescaler = indices*SPLIT_SIZE
  print(rescaler)
  print(tf.range(len(rescaler)))
  upscaler_1 = tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2], dtype=tf.int64)],axis=-1)
  print(upscaler_1)
  target_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                       repeats=[len(rescaler)], axis=0)*tf.cast(y_target_extract[...,1:5], dtype = tf.float32)
  print(target_upscaler_2)
  break

(32, 224, 224, 3) tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(
[[1.         0.7751998  0.74805427 ... 0.         0.         0.        ]
 [1.         0.7351998  0.49075517 ... 0.         0.         0.        ]
 [1.         0.3887273  0.73880965 ... 0.         0.         0.        ]
 ...
 [1.         0.52800006 0.9480001  ... 0.         0.         0.        ]
 [1.         0.2313455  0.44174176 ... 0.         0.         1.        ]
 [1.         0.47094548 0.5        ... 0.         0.         0.        ]], shape=(65, 25), dtype=float32)
tf.Tensor(
[1.         0.7751998  0.74805427 0.08552727 0.10301401 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         0.         0.         0.
 0.        ], shape=(25,), dtype=float32)
tf.Tensor(
[[ 0  1  4]
 [ 0  3  3]
 [ 0  5  4]
 [ 1  3  2]
 [ 2  1  4]
 [ 2  5  2]
 [ 3  4  3]
 [ 4  3  3]
 [ 4  3  5]
 [ 5  3  3]
 [ 6  1  2]
 [ 6  1  4]
 [ 6  3  3]

In [29]:
cv2.imwrite('out_2.jpg',np.array(i[2]))

True

# Model

In [25]:
NUM_FILTERS=512
OUTPUT_DIM=N_CLASSES+5*B

In [26]:
#  base_model=tf.keras.applications.efficientnet.EfficientNetB1(
base_model = tf.keras.applications.resnet50.ResNet50(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)
base_model.trainable=False

In [27]:
class CustomConvBlock(Layer):
  def __init__(self,filters,kernel_size,padding,kernel_initializer,is_norm=True):
    super(CustomConvBlock,self).__init__()
    self.conv = Conv2D(filters=filters,kernel_size=kernel_size,padding=padding,kernel_initializer=kernel_initializer)
    self.is_norm = is_norm
    if is_norm:
      self.norm = BatchNormalization()
    self.leaky_relu = LeakyReLU(alpha=0.1)

  def call(self,input):
    x = self.conv(input)
    if self.is_norm:
      x = self.norm(x)
    x=self.leaky_relu(x)
    return x


In [28]:
class ObjectDetector(Model):
  def __init__(self,base_model,n_layers,filters,kernel_size,padding,kernel_initializer,split_size,output_dim):
    super(ObjectDetector,self).__init__()
    self.n_layers = n_layers
    self.base_model = base_model
    self.custom_conv_blocks = [CustomConvBlock(filters,kernel_size,padding,kernel_initializer) for _ in range(n_layers)]
    self.custom_conv_block_without_norm = CustomConvBlock(filters,kernel_size,padding,kernel_initializer,is_norm=False)
    self.flatten = Flatten()
    self.dense_1 = Dense(filters,kernel_initializer='he_normal')
    self.norm = BatchNormalization()
    self.leaky_relu= LeakyReLU(alpha=0.1)
    self.dropout = Dropout(0.5)
    self.dense_2 = Dense(split_size*split_size*output_dim,activation='sigmoid')
    self.reshape = Reshape((split_size,split_size,output_dim))

  def call(self,input):
    x=self.base_model(input)
    for block in self.custom_conv_blocks:
      x = block(x)
    x = self.custom_conv_block_without_norm(x)
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.norm(x)
    x = self.leaky_relu(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    x = self.reshape(x)
    return x

  def build_graph(self):
    x = Input(shape=(H,W,3))
    return Model(inputs=[x], outputs=self.call(x))

In [29]:
model = ObjectDetector(base_model,3,NUM_FILTERS,(3,3),'same','he_normal',SPLIT_SIZE,OUTPUT_DIM)
model(np.ones(shape=(1,224,224,3)))

<tf.Tensor: shape=(1, 7, 7, 30), dtype=float32, numpy=
array([[[[0.76800174, 0.4962102 , 0.4289068 , ..., 0.43133533,
          0.49104235, 0.5257758 ],
         [0.72254825, 0.5035205 , 0.55210626, ..., 0.3628404 ,
          0.68101674, 0.4099604 ],
         [0.61019725, 0.47441834, 0.40609127, ..., 0.49260452,
          0.5254388 , 0.71720016],
         ...,
         [0.4569286 , 0.40011162, 0.659942  , ..., 0.51353693,
          0.6178691 , 0.5309931 ],
         [0.5757246 , 0.56307644, 0.5198629 , ..., 0.4006491 ,
          0.47205913, 0.3938104 ],
         [0.44510296, 0.49533448, 0.33729595, ..., 0.4755506 ,
          0.34001628, 0.36534318]],

        [[0.48429498, 0.39200938, 0.46122372, ..., 0.39857823,
          0.58371985, 0.36802697],
         [0.60549974, 0.6064813 , 0.37609676, ..., 0.485353  ,
          0.3052611 , 0.554605  ],
         [0.4830617 , 0.6353721 , 0.5209549 , ..., 0.54796445,
          0.55653703, 0.55048263],
         ...,
         [0.2982964 , 0.592142  ,

In [30]:
model.build_graph().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 custom_conv_block (CustomCo  (None, 7, 7, 512)        9439744   
 nvBlock)                                                        
                                                                 
 custom_conv_block_1 (Custom  (None, 7, 7, 512)        2361856   
 ConvBlock)                                                      
                                                                 
 custom_conv_block_2 (Custom  (None, 7, 7, 512)        2361856   
 ConvBlock)                                                      
                                                             

In [36]:
def compute_iou(bbox1,bbox2):
  bbox1_new = tf.stack([
      bbox1[...,0]-bbox1[...,2]/2,
      bbox1[...,1]-bbox1[...,3]/2,
      bbox1[...,0]+bbox1[...,2]/2,
      bbox1[...,1]+bbox1[...,3]/2
  ],axis=-1)

  bbox2_new = tf.stack([
      bbox2[...,0]-bbox2[...,2]/2,
      bbox2[...,1]-bbox2[...,3]/2,
      bbox2[...,0]+bbox2[...,2]/2,
      bbox2[...,1]+bbox2[...,3]/2
  ],axis=-1)

  inter_top_left_max = tf.maximum(bbox1_new[...,:2],bbox2_new[...,:2])
  inter_bottom_right_min = tf.minimum(bbox1_new[...,2:],bbox2_new[...,2:])
  inter_dimensions = tf.maximum(0.0,inter_bottom_right_min-inter_top_left_max)
  inter_area = inter_dimensions[...,0]*inter_dimensions[...,1]

  bbox1_area = bbox1[...,2]*bbox1[...,3]
  bbox2_area = bbox2[...,2]*bbox2[...,3]
  total_area = tf.maximum(bbox1_area+bbox2_area-inter_area,1e-10)
  return tf.clip_by_value(inter_area/total_area,0.0,1.0)

In [37]:
def square_difference(x,y):
  return tf.reduce_sum(tf.square(y-x))

In [31]:
def yolo_loss(y_true,y_pred):
  confidence = y_true[...,0]
  # object loss
  pos_of_ones = tf.where(confidence[:]==1)
  y_true_extract = tf.gather_nd(y_true,pos_of_ones)
  y_pred_extract = tf.gather_nd(y_pred,pos_of_ones)

  rescaler = pos_of_ones*SPLIT_SIZE
  upscaler = tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2], dtype=tf.int64)],axis=-1)

  y_true_upscaler = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],repeats=[len(rescaler)],axis=0)*tf.cast(y_true_extract[...,1:5], dtype = tf.float32)
  y_pred_1_upscaler = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,1:5], dtype = tf.float32)
  y_pred_2_upscaler = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,6:10], dtype = tf.float32)

  y_true_orig = tf.cast(upscaler, dtype = tf.float32)+y_true_upscaler
  y_pred_1_orig = tf.cast(upscaler, dtype = tf.float32)+y_pred_1_upscaler
  y_pred_2_orig = tf.cast(upscaler, dtype = tf.float32)+y_pred_2_upscaler

  mask = tf.cast(tf.math.greater(compute_iou(y_true_orig,y_pred_2_orig),compute_iou(y_true_orig,y_pred_1_orig)),dtype=tf.int32)

  y_pred_joined = tf.transpose(tf.stack([y_pred_extract[...,0],y_pred_extract[...,5]],axis=0))

  obj_pred = tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))

  obj_loss = square_difference(tf.cast(obj_pred,dtype =tf.float32),tf.cast(tf.ones([len(rescaler)]),dtype=tf.float32))

  # no object loss

  pos_of_zeros = tf.where(confidence[:]==0)
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5],pos_of_zeros)
  y_true_extract = tf.zeros(shape=len(y_pred_extract))

  no_obj_loss_1 = square_difference(tf.cast(y_true_extract,dtype=tf.float32),tf.cast(y_pred_extract[...,0],dtype=tf.float32))
  no_obj_loss_2 = square_difference(tf.cast(y_true_extract,dtype=tf.float32),tf.cast(y_pred_extract[...,5],dtype=tf.float32))

  no_obj_loss = no_obj_loss_1+no_obj_loss_2

  # object class loss

  y_true_extract = tf.gather_nd(y_true[...,5:],pos_of_ones)
  y_pred_extract = tf.gather_nd(y_pred[...,B*5:],pos_of_ones)

  class_loss = square_difference(tf.cast(y_pred_extract,dtype =tf.float32),tf.cast(y_true_extract,dtype=tf.float32))

  # object bbox loss

  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], pos_of_ones)
  y_true_center_extract = tf.gather_nd(y_true[...,1:3],pos_of_ones)
  y_pred_center_joined = tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=-1)
  y_pred_center_extract = tf.gather_nd(y_pred_center_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  center_loss = square_difference(tf.cast(y_pred_center_extract,dtype=tf.float32),tf.cast(y_true_center_extract,dtype=tf.float32))

  y_true_size_extract = tf.gather_nd(y_true[...,3:5],pos_of_ones)
  y_pred_size_joined = tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=-1)
  y_pred_size_extract = tf.gather_nd(y_pred_size_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  size_loss = square_difference(tf.cast(y_pred_size_extract,dtype=tf.float32),tf.cast(y_true_size_extract,dtype=tf.float32))

  box_loss = center_loss+size_loss

  # all losses added

  lambda_coord = 5.0
  lambda_no_obj = 0.5

  loss = obj_loss + (lambda_no_obj*no_obj_loss)+ tf.cast(lambda_coord*box_loss,dtype=tf.float32)+ tf.cast(class_loss,dtype=tf.float32)
  return loss


In [42]:
checkpoint_filepath='/notebooks/object_detection/yolo_resnet_50_new.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [45]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch>=40 and epoch<80:
    return 5e-4
  else:
    return 1e-4

In [46]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [33]:
model.compile(
  loss=yolo_loss,
  optimizer=Adam(1e-3),
  metrics=['accuracy'],
  # run_eagerly=True
)

In [53]:
history = model.fit(
  train_dataset,
  validation_data=val_dataset,
  verbose=1,
  epochs=30,
  callbacks = [lr_callback,model_checkpoint_callback]
)

Epoch 1/30
534/534 [==============================] - 275s 515ms/step - loss: 163.5627 - accuracy: 2.8709e-04 - val_loss: 186.8730 - val_accuracy: 3.1888e-04 - lr: 0.0010
Epoch 2/30
534/534 [==============================] - 194s 363ms/step - loss: 153.9828 - accuracy: 1.9617e-04 - val_loss: 182.1526 - val_accuracy: 9.5663e-04 - lr: 0.0010
Epoch 3/30
534/534 [==============================] - 214s 400ms/step - loss: 146.8777 - accuracy: 2.3086e-04 - val_loss: 169.9888 - val_accuracy: 6.3776e-04 - lr: 0.0010
Epoch 4/30
534/534 [==============================] - 211s 395ms/step - loss: 127.0170 - accuracy: 3.9235e-04 - val_loss: 161.0515 - val_accuracy: 0.0013 - lr: 0.0010
Epoch 8/30
534/534 [==============================] - 201s 376ms/step - loss: 123.5049 - accuracy: 4.0072e-04 - val_loss: 156.3174 - val_accuracy: 3.1888e-04 - lr: 0.0010
Epoch 9/30
534/534 [==============================] - 198s 370ms/step - loss: 119.6360 - accuracy: 4.1508e-04 - val_loss: 154.9406 - val_accuracy: 9.

# Testing the model on COCO Dataset

In [43]:
model.load_weights(checkpoint_filepath)

In [44]:
test_path = '/notebooks/test_images/'

In [45]:
def test_model(filename):
  try:
    im_path = test_path+filename
    print(im_path)
    img=cv2.resize(cv2.imread(im_path),(H,W))

    image=tf.io.decode_jpeg(tf.io.read_file(im_path))
    image=tf.image.resize(image, [H,W])
    output=model.predict(np.expand_dims(image, axis = 0))
    THRESH=.25
    object_positions=tf.concat([tf.where(output[...,0]>=THRESH),tf.where(output[...,5]>=THRESH)],axis=0)
    print(object_positions)
    selected_output=tf.gather_nd(output,object_positions)
    print(selected_output)
    final_boxes=[]
    final_scores=[]
    for i,pos in enumerate(object_positions):
      for j in range(2):      
        if selected_output[i][j*5]>THRESH:
          output_box=tf.cast(output[pos[0]][pos[1]][pos[2]][(j*5)+1:(j*5)+5],dtype=tf.float32)
          
          x_centre=(tf.cast(pos[1],dtype=tf.float32)+output_box[0])*32
          y_centre=(tf.cast(pos[2],dtype=tf.float32)+output_box[1])*32

          x_width,y_height=tf.math.abs(H*output_box[2]),tf.math.abs(W*output_box[3])
          
          x_min,y_min=int(x_centre-(x_width/2)),int(y_centre-(y_height/2))
          x_max,y_max=int(x_centre+(x_width/2)),int(y_centre+(y_height/2))

          if(x_min<=0):x_min=0
          if(y_min<=0):y_min=0
          if(x_max>=W):x_max=W
          if(y_max>=H):y_max=H
          final_boxes.append(
              [x_min,y_min,x_max,y_max,
              str(classes[tf.argmax(selected_output[...,10:],axis=-1)[i]])])
          final_scores.append(selected_output[i][j*5])
    print(final_scores)
    print('finalboxes',final_boxes)
    final_boxes=np.array(final_boxes)
    
    object_classes=final_boxes[...,4]
    nms_boxes=final_boxes[...,0:4]

    nms_output=tf.image.non_max_suppression(
        nms_boxes,final_scores,max_output_size=100,iou_threshold=0.2,
        score_threshold=float('-inf')
    )
    print(nms_output)

    for i in nms_output:
      cv2.rectangle(
          img,
          (int(final_boxes[i][0]),int(final_boxes[i][1])),
          (int(final_boxes[i][2]),int(final_boxes[i][3])),(0,0,255),1)
      cv2.putText(
          img,
          final_boxes[i][-1],
          (int(final_boxes[i][0]),int(final_boxes[i][1])+15),
          cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(2,225,155),1
          )
        
    cv2.imwrite('/notebooks/outputs/'+filename[:-4]+'_det'+'.jpg',cv2.resize(img,(384,384)))
  except:
    print('No object found')

In [68]:
for filename in os.listdir(test_path):
  test_model(filename)

/notebooks/test_images/000000000019.jpg
1/1 [==============================] - 0s 58ms/step
tf.Tensor(
[[0 3 4]
 [0 3 4]], shape=(2, 3), dtype=int64)
tf.Tensor(
[[9.0563107e-01 6.0313839e-01 6.0639292e-01 9.4996995e-01 6.9253266e-01
  8.6871964e-01 4.2168576e-01 4.6402973e-01 6.9183618e-01 5.1744831e-01
  4.4565573e-02 7.5895037e-03 2.8665433e-02 1.0029381e-01 1.8148261e-04
  2.3986049e-02 1.3480465e-02 1.3553432e-02 4.2639826e-03 9.5408279e-01
  8.6734275e-04 6.1744653e-02 9.3616769e-03 2.2212168e-02 9.7859662e-04
  2.8320111e-02 3.4004554e-02 2.0492071e-02 5.4030858e-02 2.4387757e-03]
 [9.0563107e-01 6.0313839e-01 6.0639292e-01 9.4996995e-01 6.9253266e-01
  8.6871964e-01 4.2168576e-01 4.6402973e-01 6.9183618e-01 5.1744831e-01
  4.4565573e-02 7.5895037e-03 2.8665433e-02 1.0029381e-01 1.8148261e-04
  2.3986049e-02 1.3480465e-02 1.3553432e-02 4.2639826e-03 9.5408279e-01
  8.6734275e-04 6.1744653e-02 9.3616769e-03 2.2212168e-02 9.7859662e-04
  2.8320111e-02 3.4004554e-02 2.0492071e-02 5.

# Testing in Real Time

In [46]:
cap = cv2.VideoCapture(1)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(rgb, (H,W))
    
    output=model.predict(np.expand_dims(image, axis = 0))
    THRESH=.25
    object_positions=tf.concat([tf.where(output[...,0]>=THRESH),tf.where(output[...,5]>=THRESH)],axis=0)
    print(object_positions)
    selected_output=tf.gather_nd(output,object_positions)
    print(selected_output)
    final_boxes=[]
    final_scores=[]
    for i,pos in enumerate(object_positions):
      for j in range(2):      
        if selected_output[i][j*5]>THRESH:
          output_box=tf.cast(output[pos[0]][pos[1]][pos[2]][(j*5)+1:(j*5)+5],dtype=tf.float32)
          
          x_centre=(tf.cast(pos[1],dtype=tf.float32)+output_box[0])*32
          y_centre=(tf.cast(pos[2],dtype=tf.float32)+output_box[1])*32

          x_width,y_height=tf.math.abs(H*output_box[2]),tf.math.abs(W*output_box[3])
          
          x_min,y_min=int(x_centre-(x_width/2)),int(y_centre-(y_height/2))
          x_max,y_max=int(x_centre+(x_width/2)),int(y_centre+(y_height/2))

          if(x_min<=0):x_min=0
          if(y_min<=0):y_min=0
          if(x_max>=W):x_max=W
          if(y_max>=H):y_max=H
          final_boxes.append(
              [x_min,y_min,x_max,y_max,
              str(classes[tf.argmax(selected_output[...,10:],axis=-1)[i]])])
          final_scores.append(selected_output[i][j*5])
    print(final_scores)
    print('finalboxes',final_boxes)
    final_boxes=np.array(final_boxes)
    
    object_classes=final_boxes[...,4]
    nms_boxes=final_boxes[...,0:4]

    nms_output=tf.image.non_max_suppression(
        nms_boxes,final_scores,max_output_size=100,iou_threshold=0.2,
        score_threshold=float('-inf')
    )
    print(nms_output)
    factor = 450./H
    for i in nms_output:
      cv2.rectangle(
          frame,
          (int(float(final_boxes[i][0])*factor),int(float(final_boxes[i][1])*factor)),
          (int(float(final_boxes[i][2])*factor),int(float(final_boxes[i][3])*factor)),(0,0,255),1)
      cv2.putText(
          frame,
          final_boxes[i][-1],
          (int(float(final_boxes[i][0])*factor),int(float(int(final_boxes[i][1])+15)*factor)),
          cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(2,225,155),1
          )
    cv2.imshow('Track', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()

[ WARN:2@243.746] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:2@243.746] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
